In [1]:
test_folder_path="/ssd8/2023COVID19/2023_covid/test_covid/unzip_ct"

In [2]:
import os
import gc
import cv2
import math
import copy
import time
import random
import shutil, sys    

import glob
# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2


from sklearn.metrics import f1_score,roc_auc_score


import timm
from timm.models.efficientnet import *

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict


import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import f1_score

import matplotlib.pyplot as plt
from scipy import ndimage

In [3]:
test_ct_list=list(glob.glob(os.path.join(test_folder_path, "*"))) 

In [4]:
len(test_ct_list)

4308

In [5]:
# test_ct_list=test_ct_list[-400:-390]   #quick test1
# test_ct_list=test_ct_list[-1] #quick test2

In [6]:
#quick test2
if isinstance(test_ct_list, str):
    test_ct_list=[test_ct_list]
    print("run this code")

In [7]:
def autocropmin(image, threshold=100, kernsel_size = 10):
        
    img = image.copy()
    
    SIZE = img.shape[0]
    imgfilt = ndimage.minimum_filter(img, size=kernsel_size)
    img_b=np.where(imgfilt<threshold,0,255)
    a=img_b[:,:,0].sum(axis=1)
    a=np.concatenate(([0],a,[0]))

    a_=np.where(a==0)[0]
    mina=a_[np.argmax(a_[1:]-a_[:-1])]
    maxa=a_[np.argmax(a_[1:]-a_[:-1])+1]-1

    b=img_b[:,:,0].sum(axis=0)
    b=np.concatenate(([0],b,[0]))

    b_=np.where(b==0)[0]
    minb=b_[np.argmax(b_[1:]-b_[:-1])]
    maxb=b_[np.argmax(b_[1:]-b_[:-1])+1]-1

    if  mina!=maxa and minb!=maxb:
        imageout=img[mina:maxa,minb:maxb,:]
    else:
        imageout=img

    return imageout

In [8]:
diff_shape_ct_list=[]
for i in tqdm(range(len(test_ct_list))):
    
    tmp_list=list(glob.glob(os.path.join(test_ct_list[i], "*")))
    
    
    
    tmp_shape_set=set()
    for j in range(len(tmp_list)):
    
        str1=tmp_list[j]
        img=cv2.imread(str1)
        
        try:
            tmp_shape_set.add(img.shape)
        except:
            print("bug file")
            continue
        
        img=autocropmin(img)


        str1=str1.replace(test_folder_path,"/ssd8/2023COVID19/Train_Valid_dataset/test_crop")


        folder_path="/".join(str1.split("/")[:-1])
        if len(tmp_shape_set)!=1:
            shutil.rmtree(folder_path)
            diff_shape_ct_list.append(folder_path.split("/")[-1])
            print(folder_path)
            break
            
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        cv2.imwrite(str1,img)


  5%|▍         | 200/4308 [26:18<7:07:55,  6.25s/it] 

/ssd8/2023COVID19/Train_Valid_dataset/test_crop/32d895c2-f4db-4187-afed-823a4a045367


 16%|█▌        | 670/4308 [1:34:57<9:15:10,  9.16s/it] 

bug file
bug file
bug file
bug file
bug file
bug file
bug file
bug file
bug file
bug file
bug file
bug file


 16%|█▌        | 671/4308 [1:35:06<9:03:01,  8.96s/it]

...

 99%|█████████▉| 4269/4308 [9:39:06<04:42,  7.26s/it]

/ssd8/2023COVID19/Train_Valid_dataset/test_crop/6ad1bafb-368b-46be-9ea4-829fb9bb258d


 99%|█████████▉| 4285/4308 [9:41:22<02:24,  6.28s/it]

bug file
bug file
bug file
bug file
bug file
bug file
bug file


100%|██████████| 4308/4308 [9:43:34<00:00,  8.13s/it]


In [13]:
for ct in diff_shape_ct_list:
    str1=test_folder_path+"/"+ct
    str2="/ssd8/2023COVID19/Train_Valid_dataset/test_crop/"+ct
    
    tmp_list=list(glob.glob(os.path.join(str1, "*")))
    last_file=str(len(tmp_list)-1)+".jpg"
    str1=str1+"/"+last_file
    str2=str2+"/"+last_file
    # print(str1)
    # print(str2)
    img=cv2.imread(str1)
    img=autocropmin(img)
    folder_path="/".join(str2.split("/")[:-1])
    # print(folder_path)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    cv2.imwrite(str2,img)
    

In [14]:
len(os.listdir("/ssd8/2023COVID19/Train_Valid_dataset/test_crop/"))

4308